<a href="https://colab.research.google.com/github/Souleymaneawe/Souleymaneawe/blob/main/health_platform_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import requests
import json
from google.colab import userdata

In [2]:
# Récupérer la clé API Gemini depuis les Secrets de Colab
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except userdata.SecretNotFoundError:
    GEMINI_API_KEY = None
    print("Erreur : Clé API Gemini non trouvée. Ajoutez 'GEMINI_API_KEY' dans les Secrets de Colab.")

# Configuration de l'API Gemini
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent"

# Charger les données des catégories santé à partir du fichier JSON
def load_health_categories():
    try:
        with open("health_categories.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return {
            "categories": [
                {"name": "Alimentation", "description": "Conseils pour une alimentation saine et équilibrée."},
                {"name": "Exercice Physique", "description": "Recommandations pour l'activité physique et le fitness."},
                {"name": "Bien-être Mental", "description": "Stratégies pour gérer le stress et améliorer la santé mentale."}
            ]
        }

# Fonction pour appeler l'API Gemini
def get_gemini_response(prompt):
    if not GEMINI_API_KEY:
        return "Erreur : Clé API Gemini manquante."
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "contents": [{"parts": [{"text": prompt}]}]
    }
    try:
        response = requests.post(f"{GEMINI_API_URL}?key={GEMINI_API_KEY}", headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        return result.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "Erreur de réponse de l'API")
    except requests.exceptions.RequestException as e:
        return f"Erreur API : {str(e)}"

# Fonction pour les recommandations santé
def get_health_recommendation(category, user_input):
    prompt = f"Donne une recommandation personnalisée pour la catégorie {category}. Contexte de l'utilisateur : {user_input}."
    return get_gemini_response(prompt)

# Fonction pour répondre aux questions santé
def answer_health_question(question):
    prompt = f"Réponds à cette question sur la santé : {question}. Fournis une réponse concise et utile."
    return get_gemini_response(prompt)

# Interface Gradio
with gr.Blocks(title="Plateforme de Suivi de la Santé avec Gemini") as app:
    gr.Markdown("# Plateforme de Suivi de la Santé avec Gemini")

    # Charger les catégories santé
    health_data = load_health_categories()
    category_names = [category["name"] for category in health_data["categories"]]

    # Section pour les catégories santé
    gr.Markdown("## Catégories Santé")
    category = gr.Dropdown(choices=category_names, label="Choisissez une catégorie")
    category_description = gr.Textbox(label="Description de la catégorie", interactive=False)

    # Mettre à jour la description de la catégorie sélectionnée
    def update_category_description(selected_category):
        for category_data in health_data["categories"]:
            if category_data["name"] == selected_category:
                return category_data["description"]
        return "Catégorie non trouvée."

    category.change(fn=update_category_description, inputs=category, outputs=category_description)

    # Recommandation santé
    gr.Markdown("## Recommandation Personnalisée")
    user_input = gr.Textbox(label="Décrivez votre situation (ex. mode de vie, objectifs)")
    recommend_button = gr.Button("Obtenir une Recommandation")
    recommendation_output = gr.Textbox(label="Recommandation")
    recommend_button.click(fn=get_health_recommendation, inputs=[category, user_input], outputs=recommendation_output)

    # Questions santé
    gr.Markdown("## Posez une Question Santé")
    question = gr.Textbox(label="Votre question")
    question_button = gr.Button("Obtenir une Réponse")
    question_output = gr.Textbox(label="Réponse")
    question_button.click(fn=answer_health_question, inputs=question, outputs=question_output)

In [5]:
# Exécutez l'application Gradio avec share=True
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://288ecfe4c5a44aa678.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
